# **<span style='color:orange'>Preprocesamiento</span>**

### Aqui se realiza el análisis estadístico exploratorio y el preprocesamiento de los datos.

**El objetivo final es preparar el conjunto de datos para la aplicación de algoritmos de Machine Learning**

#### Exploración estadística 

In [ ]:
#Importamos las herramientas necesarias

import numpy as np # funciones matemáticas
import pandas as pd # herramientas de analisis de datos
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OrdinalEncoder #herrmaienta para convertir a valores enteros
from sklearn.preprocessing import OneHotEncoder #herramienta para binarizar variables
import seaborn as sns # Visualization
import matplotlib.pyplot as plt

In [ ]:
#importamos los datos y los leemos con la función pd.read_excel.
#De ahora en adelante cada vez que declaremos df estaremos haciendo referencia a nuestros datos

df = pd.read_excel("./DATA/fragmentos.xlsx",sheet_name='dset', engine='openpyxl')


In [ ]:
df.info() #información básica  sobre el dataset

Exploremos la información que el comando **df.info()** nos brinda:

* Tenemos 578 entradas que corresponden al número de fragmentos númerados en un indice del 0 al 577
* Tenemos un total de 22 columnas (numeradas de 0 a 21) que representan cada una de las caracteristicas con las que describimos los fragmentos.
* En esta tabla resúmen, la columna Non-Null count nos informa sobre el número de valores faltantes en cada columna. Como observamos no tenemos valores faltantes en nuestros datos. 
* La columna Dtype nos informa sobre el tipo de datos que contiene nuestra matriz, el tipo **int64** corresponde a  vaiables numéricas mientras que **object** se refiere a variables categóricas. Como podemos observar la nuestra es una **matriz de datos mixta** con 19 variables categóricas y 2 numéricas

**El proximo paso es convertir las columnas categóricas en numéricas**

Para esto vamos a separar separar las variables ordinales de las nominales para aplicar diferentes procedimientos.

**```Recordemos que las variables ordinales tienen una relación de orden mientras que las nominales solo describen en el fondo la presencia o ausencia de una caracteristica```**


In [ ]:
datos_numericos = df.select_dtypes(include=[np.number])
datos_numericos.info()

**El siguiente paso es convertir las columnas categóricas en numéricas y simplificar tanto como sea posible nuestro espacio predictor, es decir eliminar las columnas que no son realmente importantes para la clasificación cerámica**

Exploremos un momento la variable decoración:

In [ ]:
#df['Elemento'].value_counts()
df['Decoracion'].value_counts()

In [ ]:
 (df['Decoracion'].value_counts()['Ausente'] / df.shape[0]) * 100

**557** fragmentos, es decir el **96,36%** de los fragmentos no tienen ninguna decoración, por lo tanto no es posible utilizar las variables decorativas en nuestro problema debido a que no tenemos datos suficientes para considerar su información representativa.
Tampoco vamos a utilizar las variables relativas al color porque queremos reducir al maximo posible los sesgos producto de la percepción de los analístas. El color es muy susceptible de ser percibido de forma diferente por diferentes personas asi que eliminaremos toda la información relativa al color de los fragmentos

In [ ]:
#eliminamos algunas columnas que no usaremos en nuestro ejercicio
df = df.drop(['Color_Eng _o_baño2', 'Color_Baño', 'Color_Engobe', 'Color_Munsell', 'Decoracion',
             'Tecnica', 'Elemento₁', 'Localizacion' ], axis=1)
df.info()

Ya tenemos un primer filtro de nuestra matriz de datos. **El proximo paso es convertir las columnas categóricas en numéricas**

Para esto vamos a separar separar las variables ordinales de las nominales para aplicar diferentes procedimientos.

**```Recordemos que las variables ordinales tienen una relación de orden mientras que las nominales solo describen la presencia o ausencia de una caracteristica```**


In [49]:
ordinales = ['T_desgrasante', 'Densidad', 'Acabado_Interno', 'Acabado_Externo',
               'Dureza'] #seleccionamos las variables ordinales y las agrupamos en una nueva tabla

nominales = df.columns.difference(ordinales) # Hacemos lo mismo con las nominales

variables_ordinales = df[ordinales]
variables_nominales = df[nominales]         
variables_ordinales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 0 to 577
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   T_desgrasante    578 non-null    object
 1   Densidad         578 non-null    object
 2   Acabado_Interno  578 non-null    object
 3   Acabado_Externo  578 non-null    object
 4   Dureza           578 non-null    object
dtypes: object(5)
memory usage: 22.7+ KB


In [62]:
orden_T_desgrasante = ['Muy  fino', 'Fino', 'Medio', 'Grueso']
orden_Densidad = ['Equilibrado', 'Mas desgrasante']
orden_Acabado_Interno = ['Burdo', 'Erosionado','Fino','Pulido']
orden_Acabado_Externo = ['Burdo', 'Erosionado', 'Fino', 'Pulido']
orden_Dureza = ['Raya con la uña', 'Dura', 'Muy dura']



[{'Fino': 0, 'Grueso': 1, 'Medio': 2, 'Muy  fino': 0},
 {'Equilibrado': 0, 'Mas desgrasante': 1},
 {'Burdo': 0, 'Erosionado': 1, 'Fino': 2, 'Pulido': 3}, 
 {'Burdo': 0, 'Erosionado': 1, 'Fino': 2, 'Pulido': 3},
 {'Dura': 0, 'Muy dura': 1, 'Raya con la uña': 2}]

    
categories = [orden_T_desgrasante , orden_Densidad, orden_Acabado_Interno,
orden_Acabado_Externo, orden_Dureza ]
              

In [64]:

enc = OrdinalEncoder(dtype= 'int64', categories = categories) #estamos usando una de las herramientas importadas en el primer paso

enc.fit_transform(variables_ordinales)
variables_ordinales_codificadas = pd.DataFrame(enc.fit_transform(variables_ordinales),columns= variables_ordinales.columns)
variables_ordinales_codificadas # nuestra tabla transformada a enteros 


,T_desgrasante,Densidad,Acabado_Interno,Acabado_Externo,Dureza
0,2,1,2,2,0
1,2,0,2,2,0
2,2,0,2,2,0
3,2,1,1,1,1
4,1,0,1,2,0
...,...,...,...,...,...
573,1,0,1,2,0
574,1,0,1,1,0
575,1,0,2,2,0
576,1,0,2,2,1


In [ ]:
variables_ordinales_codificadas # nuestra tabla transformada a enteros 

In [ ]:
variables_ordinales_codificadas['Dureza'][577]

In [ ]:
etiquetas = enc.categories_
etiquetas

In [65]:
encoding = enc.categories_
encoding_feature = lambda variables_ordinales_codificadas: dict(zip(variables_ordinales_codificadas,
                                                           range(len(variables_ordinales_codificadas))))
encoding_full = [encoding_feature(feature_elem) for feature_elem in encoding ]
print(encoding_full)

[{'Muy  fino': 0, 'Fino': 1, 'Medio': 2, 'Grueso': 3}, {'Equilibrado': 0, 'Mas desgrasante': 1}, {'Burdo': 0, 'Erosionado': 1, 'Fino': 2, 'Pulido': 3}, {'Burdo': 0, 'Erosionado': 1, 'Fino': 2, 'Pulido': 3}, {'Raya con la uña': 0, 'Dura': 1, 'Muy dura': 2}]


### Métodos para explorar la correlación y multicolinearidad entre las variables

Una vez eliminadas las columnas que **sabemos** que no nos sirven, vamos a usar dos procedimientos diferentes para intentar simplificar aún más nuestro problema

* [Matrices de correlación](https://es.wikipedia.org/wiki/Matriz_de_correlaci%C3%B3n)
* [Factor de inflación de la varianza](https://es.wikipedia.org/wiki/Factor_de_inflaci%C3%B3n_de_la_varianza)

Las matrices de correlación permiten una inspección visual intuitiva del comportamiento de las variables, veamos:

In [ ]:
#Matrices de correlación

correlation_matrix = df.corr(method='spearman') #Mediante el parametro method= se puede escoger entre varios
                                                #coeficientes de correlación {‘pearson’,
                                                #‘kendall’, ‘spearman’}

sns_plot = sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm',fmt=".1f") 
fig = sns_plot.get_figure()
#fig.savefig("./Un-modelo-de-juguete/figures/correlation.png", bbox_inches='tight')
plt.show()

Nos interesa fijarnos en aquellas relaciones más extremas entre las variables. Aquellas que se comportan de manera similar decimos que tienen una alta correlación positiva y quiere decir que valores altos en una variable estan acompañados de valores altos en la otra, estan representados por un color rojo intenso. 
Observemos  la relación entre las variables 'Elemento₁', 'Localizacion', y 'Tecnica' que conforman ese recuadro rojo con valores altos que vemos

In [ ]:
# Vamos a construir una función para calcular el Factor de inflación de la varianza (VIF)

def calc_vif(df):
    # Calculando el VIF
    vif = pd.DataFrame() # creamos la tabla para visualizar los resultados
    vif["variables"] = df.columns # la columna 'variables' va a tener el nombre de las columnas de nuestra matriz
    vif["VIF"] = [variance_inflation_factor(df.values, i) 
                  for i in range(df.shape[1])] # calculamos el VIF para cada columna

    return(vif)

La intuición detras del computo del VIF es la eliminación de aquellas variables con puntajes muy altos, particularmente valores >= 10 indican una alta [multicolinealidad](https://es.wikipedia.org/wiki/Multicolinealidad).

Este es un ejercicio iterativo, es decir, Calculamos el VIF, eliminamos aquellas columans con el mayor valor y volvemos a calcular...

**[aqui](https://www.analyticsvidhya.com/blog/2020/03/what-is-multicollinearity/) un recurso útil para entender el funcionamiento de este método**

In [ ]:
# Aplicamos la función que creamos sobre nuestros datos df
calc_vif(df)

In [ ]:
df= df.drop(['Acabado_Externo', 'Acabado_Interno', 'Tamaño(cm)'], axis=1)
calc_vif(df)

In [ ]:
#Matrices de correlación

correlation_matrix = df.corr(method='spearman') #Mediante el parametro method= se puede escoger entre varios
                                                #coeficientes de correlación {‘pearson’,
                                                #‘kendall’, ‘spearman’}

sns_plot = sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm',fmt=".1f") 
fig = sns_plot.get_figure()
#fig.savefig("/home/dsg/Escritorio/CCA2024/figures/correlation.png", bbox_inches='tight')
plt.show()

Los mapas de calor permiten la inspección visual de variables altamente correlacionadas.
Su exclusión del conjunto de datos debe estar justificada en términos del objetivo

In [ ]:
df.to_csv('./intermediate_data/datos_preprocesados.csv')


Una vez los datos estan preprocesados, podemos realizar la [Fase No-Supervisada](./CCA_Clustering.ipynb)